In [ ]:
import dash
from dash import dcc, html, Input, Output, State
import json
import numpy as np
from tensorflow import keras
import pickle
import dash_bootstrap_components as dbc
import pycountry
import spacy
import re
from collections import Counter

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def country_in_query(query):
    for country in pycountry.countries:
        if country.name.lower() in query.lower():
            return country.name
    return None

def get_keyword_response(user_input, keyword_responses):
    # Normalize the user input
    normalized_input = user_input.lower()

    # Tokenize the user input
    input_tokens = Counter(re.findall(r'\w+', normalized_input))

    best_match = None
    best_score = 0

    for phrase, response in keyword_responses.items():
        # Tokenize the phrase
        phrase_tokens = Counter(re.findall(r'\w+', phrase))
        
        # Calculate the match score
        common_tokens = input_tokens & phrase_tokens
        score = sum(common_tokens.values())

        # Update best match if this score is higher
        if score > best_score:
            best_match = response
            best_score = score

    if best_match:
        return best_match

    return "I'm sorry, I don't understand. Can you please rephrase?"

intro_message = html.Div([
    html.H3("Welcome to the UK Student Guide Chatbot!", style={'color': '#007bff', 'textAlign': 'center'}),
    html.P("I'm here to help you with information and guidance on various topics, including:", style={'textAlign': 'center'}),
    dbc.ListGroup([
        dbc.ListGroupItem("Accommodation - Finding places to stay"),
        dbc.ListGroupItem("Travel and Transportation - Tips on traveling in the UK"),
        dbc.ListGroupItem("Healthcare - Accessing health services"),
        dbc.ListGroupItem("Jobs - Part-time work and employment tips"),
        dbc.ListGroupItem("Emergency Services - What to do in emergencies"),
        dbc.ListGroupItem("Shopping - Where to shop smart"),
    ], flush=True),
    html.P("Just type your question below to get started!", style={'textAlign': 'center', 'marginTop': '10px'}),
], style={'marginTop': '15px', 'padding': '15px', 'border': '1px solid #ddd', 'borderRadius': '5px', 'backgroundColor': '#f8f9fa'})

# Keyword-response mapping
keyword_responses = {
    
    # 1. Accommodation
    "lodging": "lodging in the uk varies from budget hostels to luxury hotels, suitable for short-term stays and convenient for travelers.",
    "place to stay": "places to stay in the uk include bed and breakfasts, guest houses, and airbnb options, offering a home-like experience.",
    "student housing": "student housing in the uk offers options like university dorms, private student halls, and shared houses, often close to campuses.",
    "rental options": "rental options in the uk range from studio apartments to family homes, available through estate agents or websites like rightmove and zoopla.",
    "living arrangements": "living arrangements for students can vary from single rooms in shared flats to private studio apartments, depending on budget and preferences.",
    "dormitory": "dormitories in uk universities provide a communal living experience, often including facilities like shared kitchens, lounges, and study areas.",
    "hostel": "hostels in the uk are a budget-friendly option, perfect for travelers and students, offering shared rooms and communal facilities.",
    "apartment": "apartments in urban areas of the uk offer modern living spaces, with options ranging from compact studios to spacious multi-bedroom units.",
    "flat": "flats in the uk, similar to apartments, are common in residential areas, available in various sizes and often with lease agreements of six months or more.",
    "room for rent": "rooms for rent are available in shared houses or flats, offering a more affordable option for students and young professionals.",
    "tell me about student accommodation": "student accommodation options in the uk include university dormitories, private apartments, and shared housing. it's recommended to plan and book your accommodation in advance, especially for popular choices.",
    "housing options in the uk": "you have a choice between on-campus dormitories provided by universities or off-campus housing, such as private apartments or shared houses. it's essential to explore your options and make arrangements early to secure suitable housing.",
    "where should i stay?": "the choice of where to stay depends on your preferences and budget. consider factors like proximity to your university, affordability, and the type of living experience you desire. on-campus dorms offer convenience, while off-campus housing provides more independence.",
    "tell me about student accommodation": "student accommodation options in the uk include university dormitories, private apartments, and shared housing. it's recommended to plan and book your accommodation in advance, especially for popular choices.",
    "housing options in the uk": "you have a choice between on-campus dormitories provided by universities or off-campus housing, such as private apartments or shared houses. it's essential to explore your options and make arrangements early to secure suitable housing.",
    "where should i stay?": "the choice of where to stay depends on your preferences and budget. consider factors like proximity to your university, affordability, and the type of living experience you desire. on-campus dorms offer convenience, while off-campus housing provides more independence.",
    "can you explain the process of booking university dorms?": "booking university dorms typically involves applying through your university's accommodation office. it's essential to follow their guidelines and deadlines.",
    "what are the advantages of living in university dorms?": "living in university dorms can offer convenience, proximity to campus, and the opportunity to meet fellow students. it's a good choice for those looking for a structured living environment.",
    "what are the benefits of off-campus housing?": "off-campus housing provides more independence and flexibility in choosing your living arrangements. it can also offer a wider range of options in terms of housing types and locations.",
    "do universities provide guidance for finding off-campus housing?": "many universities offer resources and guidance for finding off-campus housing, including information on local rental markets and support in your housing search.",
    "what are the popular student accommodation websites in the uk?": "several websites and platforms, such as rightmove, zoopla, spareroom, and gumtree, can help you find off-campus accommodation in the uk.",
    "what should i consider when budgeting for accommodation in the uk?": "when budgeting for accommodation, consider not only rent but also utilities, transportation costs, and potential additional expenses like groceries and internet services.",
    "what are the average housing prices in major uk cities?": "housing prices can vary significantly between cities in the uk. some cities like london and edinburgh tend to have higher living costs, while others may be more affordable.",
    "how can i find affordable rentals in expensive cities like london?": "to find affordable rentals in expensive cities, consider looking for accommodations in less central areas, sharing with roommates, and exploring nearby towns with lower living costs.",
    "are there any government housing schemes or subsidies for students in the uk?": "in the uk, there are various housing schemes and subsidies available to students, such as the student loans company's maintenance loan, which can help cover living expenses.",
    "what are the pros and cons of renting from a private landlord versus a letting agency?": "renting from a private landlord may offer more flexibility, while letting agencies can provide professional support. consider your preferences and needs when making a choice.",
    "what are the common rental contract lengths in the uk?": "rental contract lengths in the uk can vary, with options ranging from short-term agreements to longer leases. it's essential to understand the terms before signing.",
    "what documents and references do landlords typically require when renting a property?": "landlords often ask for documents like proof of identity, proof of income, and references from previous landlords or employers when renting a property.",
    "what are the typical upfront costs when renting a property in the uk?": "upfront costs for renting a property may include a security deposit, the first month's rent, and potential agency fees. it's essential to budget for these expenses.",
    "are there any resources for international students looking for accommodation in the uk?": "international students can often find resources and support through their universities, including guidance on accommodation options and local housing laws.",
    "are there any hostels that offer long-term stays in the uk?": "some hostels in the uk offer long-term stay options, especially in tourist-heavy areas. contact hostels directly or check hostel booking websites for availability.",
    "what's the process for subletting a room or apartment in the uk?": "subletting a room or apartment usually requires the consent of the primary leaseholder and the landlord. ensure you understand the legal aspects and responsibilities before subletting.",
    "finding accommodation with disability-friendly features in the uk": "if you have specific disability-related needs, contact your university's disability support services or explore websites specializing in accessible accommodation to find suitable options.",
    "how do i check the safety and security of a neighborhood before renting?": "you can check the safety and security of a neighborhood by researching crime statistics online, reading reviews, and asking locals or university staff for recommendations.",
    "exploring options for temporary accommodation upon arrival in the uk": "if you need temporary accommodation upon arrival, consider booking a hotel, airbnb, or hostel in advance. some universities also offer temporary housing for incoming students.",
    "what's the process for renewing a lease on an apartment or flat in the uk?": "to renew a lease, communicate with your landlord well in advance of the lease expiration date. discuss terms, potential rent changes, and renewal paperwork.",
    "finding accommodation with specific cultural or dietary preferences in mind": "if you have specific cultural or dietary preferences, look for accommodations in areas with diverse communities or inquire with your university's international student services for recommendations.",
    "how to handle roommate conflicts or issues in shared student housing": "effective communication is key. address roommate conflicts calmly and discuss issues openly. if necessary, involve your housing provider or university's housing office to mediate.",
    "exploring options for pet-friendly accommodation in the uk": "finding pet-friendly accommodation can be challenging but not impossible. look for listings that explicitly state they accept pets, and be prepared to pay additional pet deposits or fees.",
    "how do i terminate a rental agreement if i need to leave before the lease ends?": "terminating a rental agreement early may involve penalties. review your lease agreement and discuss your situation with your landlord to understand the terms and potential costs.",
    "tips for budgeting and managing housing expenses effectively": "budgeting for housing involves more than just rent. account for utilities, groceries, transportation, and miscellaneous expenses in your monthly budget.",
    "how to handle property maintenance and reporting issues in rented accommodation": "report property maintenance issues promptly to your landlord or property management. document issues with photos if needed and maintain open communication for timely resolutions.",
    "exploring co-living or co-housing options in the uk": "co-living and co-housing arrangements are becoming more popular. search for co-living spaces or groups in your area that align with your preferences and values.",
    "how to find accommodation that offers a quiet and conducive study environment": "if you prioritize a quiet study environment, consider accommodations in residential areas away from noisy nightlife districts. reading reviews can also provide insights into noise levels.",
    "how to find cheap student housing in the UK?": "Look for university-provided accommodations, shared housing options, or consider budget-friendly hostels or guest houses.",
    "what are the average monthly rents for student apartments in London?": "Average monthly rents in London vary but can be higher. Expect to pay more in central areas and less in suburbs.",
    "are there any all-female or all-male student accommodations in the UK?": "Yes, some accommodations offer all-female or all-male options for students who prefer single-gender housing.",
    "what's the best time to start looking for student housing in the UK?": "Start looking for student housing several months in advance, as popular options can fill up quickly.",
    "what are the typical lease durations for student housing in the UK?": "Lease durations for student housing can vary, but many offer options for academic year contracts or shorter terms.",
    "how do I pay for my student accommodation in the UK?": "Payment methods for student accommodation can include online transfers, direct debit, or credit/debit card payments. Check with your housing provider.",
    "what are the rules for guests staying in my student accommodation?": "Rules for guests in student accommodation vary but often require notifying roommates or housing management and limiting overnight stays.",
    "how to ensure the safety of my belongings in shared student housing?": "Ensure your belongings' safety by using locks on doors and securing valuables. Consider renter's insurance for added protection.",
    "can international students get housing on campus in the UK?": "Yes, many UK universities offer on-campus housing options for international students. Contact your university's accommodation office for details.",
    "what should I do if there's a maintenance issue in my rented accommodation?": "Report maintenance issues to your landlord or property management immediately to ensure prompt repairs.",
    "how to find accommodation near my university in a new city?": "Search online for accommodation near your university, ask for recommendations from fellow students, or contact your university's accommodation office.",
    "what's the process for moving out of student housing at the end of the lease?": "Moving out typically involves cleaning your space, returning keys, and settling any outstanding bills with the housing provider.",
    "how to avoid rental scams when searching for student accommodation?": "Avoid rental scams by dealing with reputable landlords or housing agencies. Never send money without seeing the property first.",
    "what to do if I'm unhappy with my student accommodation?": "If you're unhappy, discuss your concerns with your housing provider or university's accommodation office to explore possible solutions.",
    "how can i find nearby grocery stores or supermarkets from my accommodation?": "use map apps or ask locals or fellow students for the nearest grocery options.",
    "what's the process for requesting repairs in my rented flat?": "contact your landlord or property manager and describe the issue for repair.",
    "how do i set up internet and utilities in my student accommodation?": "check with your housing provider for guidance on setting up internet and utilities.",
    "are there laundry facilities in most student accommodations in the uk?": "many student accommodations offer on-site laundry facilities for residents.",
    "what's the policy for bringing and keeping a bicycle in student housing?": "check with your housing provider about bike storage and policies for keeping a bicycle.",
    "how to handle noisy neighbors or disruptions in shared housing?": "discuss the issue with your roommates or report disturbances to housing management if necessary.",
    "can i personalize or decorate my student room or apartment?": "most accommodations allow some personalization, but check with your housing provider for specific rules.",
    "how do i renew my lease for another semester or academic year?": "contact your landlord or housing office to inquire about lease renewal procedures.",
    "what's the process for moving out and returning keys at the end of my lease?": "follow move-out instructions provided by your housing provider, and return keys as directed.",
    "how do i ensure the safety of my personal belongings in shared spaces?": "use lockable storage options and consider renter's insurance for added protection.",
    "what's the policy for having a pet in my student accommodation?": "most student accommodations have a no-pet policy, but it's best to check with your housing provider for their specific rules.",
    "how can i find a roommate if i want to share a flat or apartment?": "use roommate finder websites/apps or check bulletin boards at your university for roommate search notices.",
    "what's the protocol for handling emergencies or maintenance issues during weekends or holidays?": "check with your housing provider for emergency contact information and procedures during off-hours.",
    "how can i pay my rent and other housing expenses while studying in the uk?": "explore payment methods such as online transfers, direct debit, or credit/debit card payments with your housing provider.",
    "what's included in the rent for my student accommodation?": "check your lease agreement or ask your housing provider for details on what's included in your rent.",
    "how can i extend my lease if i want to stay longer in my current accommodation?": "contact your housing provider to discuss lease extension options and availability.",
    "what's the procedure for getting a parking permit for my vehicle near my student accommodation?": "contact your housing provider or check with local authorities for information on obtaining a parking permit.",
    "how to handle conflicts with roommates or flatmates in shared housing?": "open communication is key; talk to your roommates about any issues and consider involving housing management if necessary.",
    "what's the policy on overnight guests or visitors in student accommodation?": "review your accommodation's guest policy, as it can vary. be respectful of roommates and neighbors when having visitors.",
    "how do i ensure the security of my accommodation and personal safety?": "lock doors and windows, avoid sharing personal information, and report any security concerns to your housing provider or local authorities.",
    "what should i do if i lose my room key or access card to my student accommodation?": "contact your housing provider immediately to report the loss and request a replacement key or card.",
    "how to deal with heating and heating bills in my accommodation during winter months?": "learn how to control heating in your space and discuss heating bills with your housing provider or flatmates.",
    "what's the policy on subletting or letting a friend stay in my room while i'm away?": "check your lease agreement and discuss subletting or guest stays with your housing provider to ensure compliance.",
    "how can i request accommodations for specific dietary needs or allergies in shared housing?": "communicate your dietary needs or allergies with your roommates and consider labeling food items in shared spaces.",
    "what's the process for getting a refund of my security deposit when i move out?": "follow the move-out instructions provided by your housing provider to ensure the return of your security deposit.",
    "how to handle issues with pests or maintenance problems in my accommodation?": "report pest issues or maintenance problems promptly to your housing provider for resolution.",
    "what's the policy on smoking or using electronic cigarettes in student accommodation?": "review your accommodation's smoking policy, which may include designated smoking areas or a complete ban.",
    "how to find local transportation options and bus routes near my student accommodation?": "use transportation apps or check with your housing provider for information on nearby bus routes and stops.",
        
    # 2. Transportation
    "public transit": "public transit in the uk includes comprehensive bus services, national and local train networks, and trams in some cities.",
    "commuting": "for daily commuting, many uk cities offer season tickets and travel cards for public transport, as well as park-and-ride schemes in busy areas.",
    "buses and trains": "buses and trains form the backbone of uk public transport, with extensive routes covering cities and rural areas, including night bus services in major cities.",
    "getting around": "getting around the uk is convenient with an integrated public transport system, contactless fare payments, and real-time travel apps.",
    "local transport": "local transport in uk towns and cities often includes buses, light rail services, and local commuter trains, with bicycle hire schemes gaining popularity.",
    "subway": "the subway system, particularly the london underground ('the tube'), is an efficient way to navigate major cities, offering fast and frequent services.",
    "metro": "metro systems, like the tyne and wear metro and london's dlr, provide rapid transit in urban areas, connecting suburbs with city centers.",
    "cycling": "cycling in uk cities is supported by dedicated bike lanes, bike-sharing schemes like santander cycles in london, and safe storage facilities.",
    "taxis": "taxis, including the iconic london black cabs, are available throughout the uk, with licensed minicabs offering pre-booked services.",
    "ride services": "ride services like uber and bolt operate in many uk cities, offering convenient app-based booking and often competitive pricing compared to traditional taxis.",
    "how to get around in the uk?": "the uk offers a range of transportation options, including buses, trains, trams, and the london underground. consider getting an oyster card or contactless payment for convenience.",
    "public transportation in the uk": "using public transport is a convenient and cost-effective way to get around the uk. familiarize yourself with local transit systems and routes.",
    "transportation options": "in the uk, you can choose from various transportation options such as buses, trains, trams, and taxis. each city may have its own public transit system, so it's useful to research the options available in your area.",
    "what are the main public transportation providers in the uk?": "the uk has several major transportation providers, including national rail, transport for london (tfl), and local bus companies. understanding these providers can help you plan your journeys.",
    "how do oyster cards work, and where can i get one?": "oyster cards are contactless payment cards used for travel in london. you can get one at oyster ticket stops, online, or at stations. they offer discounted fares compared to paper tickets.",
    "what is the london underground, and how does it operate?": "the london underground, also known as the tube, is a comprehensive metro system in london. it consists of several lines that connect the city, making it easy to navigate.",
    "what are the different types of train tickets available for travel within the uk?": "train tickets in the uk vary by type, including standard, off-peak, and advance tickets. each type offers different pricing and flexibility options for travelers.",
    "how can i save money on transportation in the uk?": "to save money on transportation, consider options like purchasing weekly or monthly travelcards, using railcards for discounts, and exploring bike-sharing services or walking for short distances.",
    "what are the options for long-distance travel within the uk?": "for long-distance travel, you can use intercity trains, coaches, or domestic flights, depending on your destination and preferences.",
    "what is the best way to plan my journeys and check public transport schedules in the uk?": "you can plan your journeys and check schedules using online resources, mobile apps, or consulting station staff for guidance.",
    "what are the accessibility options available in uk public transport for individuals with disabilities?": "uk public transport systems offer accessibility options such as ramps, priority seating, and assistance for individuals with disabilities. familiarize yourself with these services if needed.",
    "are there any student discounts or travel cards available for students in the uk?": "many universities and cities in the uk offer student travel cards or discounts on public transportation. check with your institution and local authorities for available options.",
    "what are the peak and off-peak travel times for public transport in the uk?": "peak travel times typically involve higher fares and crowded services. off-peak times offer reduced fares and a more relaxed travel experience. be aware of these timings when planning your journeys.",
    "tell me about railcards": "railcards offer discounts on train travel in the uk. they can save you money. you can buy one at a train station or online.",
    "what is a railcard?": "a railcard is a discount card for train travel. it's a great way to save on your train fares in the uk.",
    "how can i get a railcard?": "to get a railcard, visit a train station or the official website. you'll need a valid id and a photo.",
    "railcard information": "different types of railcards are available, including for students, seniors, and families, offering discounts on most rail fares.",
    "how to get railcards?": "railcards typically last for one year and can be renewed. they offer significant savings, especially for frequent travelers.",
    "what types of railcards are available?": "to apply for a railcard, you need proof of eligibility, like age or student status, and you can apply online or at train stations.",
    "can students get a discount with a railcard?": "most railcards are valid across different train networks in the uk, making them a flexible option for regular travel.",
    "where do i apply for a railcard?": "family & friends railcard offers great savings for groups, including children, ideal for family trips.",
    "what are the benefits of having a railcard?": "railcards provide significant discounts on train fares, making travel more affordable and cost-effective.",
    "are there any age restrictions for railcards?": "yes, some railcards have age restrictions. for example, the senior railcard is available to those aged 60 and over.",
    "what's the price of a railcard?": "the price of a railcard varies depending on the type and duration. it's best to check the official website for current pricing.",
    "do railcards cover all train companies?": "most railcards are valid for discounts on fares from major train companies in the uk, but it's essential to check for specific coverage.",
    "how long is a railcard valid?": "railcards typically last for one year and can be renewed for additional years of savings.",
    "can i use a railcard for all my train journeys?": "in general, railcards can be used for most train journeys in the uk, but there may be some exceptions, so it's wise to check before traveling.",
    "is there a family railcard available?": "yes, the family & friends railcard is designed for families and offers discounts for adults and children traveling together.",
    "what documents do i need to apply for a railcard?": "you'll typically need proof of eligibility, such as a valid id or documentation showing your student status or age.",
    "are there any special railcards for seniors?": "yes, the senior railcard is available for individuals aged 60 and over, offering discounted fares on train travel.",
    "how do i renew my railcard?": "you can renew your railcard online through the official railcard website, and the process is generally straightforward.",
    "can i get a refund on my railcard?": "refund policies for railcards may vary, so it's best to check the specific terms and conditions associated with your railcard purchase.",
    "railcard options for frequent travelers?": "frequent travelers can benefit from railcards as they offer significant savings on train fares, especially for those who travel regularly.",
    "what are the main public transportation providers in the uk?": "the uk has several major transportation providers, including national rail, transport for london (tfl), and local bus companies. understanding these providers can help you plan your journeys.",
    "how do oyster cards work, and where can i get one?": "oyster cards are contactless payment cards used for travel in london. you can get one at oyster ticket stops, online, or at stations. they offer discounted fares compared to paper tickets.",
    "what is the london underground, and how does it operate?": "the london underground, also known as the tube, is a comprehensive metro system in london. it consists of several lines that connect the city, making it easy to navigate.",
    "what are the different types of train tickets available for travel within the uk?": "train tickets in the uk vary by type, including standard, off-peak, and advance tickets. each type offers different pricing and flexibility options for travelers.",
    "how can i save money on transportation in the uk?": "to save money on transportation, consider options like purchasing weekly or monthly travelcards, using railcards for discounts, and exploring bike-sharing services or walking for short distances.",
    "what are the options for long-distance travel within the uk?": "for long-distance travel, you can use intercity trains, coaches, or domestic flights, depending on your destination and preferences.",
    "what is the best way to plan my journeys and check public transport schedules in the uk?": "you can plan your journeys and check schedules using online resources, mobile apps, or consulting station staff for guidance.",
    "what are the accessibility options available in uk public transport for individuals with disabilities?": "uk public transport systems offer accessibility options such as ramps, priority seating, and assistance for individuals with disabilities. familiarize yourself with these services if needed.",
    "are there any student discounts or travel cards available for students in the uk?": "many universities and cities in the uk offer student travel cards or discounts on public transportation. check with your institution and local authorities for available options.",
    "what are the peak and off-peak travel times for public transport in the uk?": "peak travel times typically involve higher fares and crowded services. off-peak times offer reduced fares and a more relaxed travel experience. be aware of these timings when planning your journeys.",
    "how do i get a student discount on public transportation in the uk?": "apply for a student discount card, such as the student oyster card in london, to access discounted fares on buses and trains.",
    "what's the best way to navigate the public transportation system in the uk?": "use transportation apps or websites to plan routes, and familiarize yourself with local bus and train schedules.",
    "how can i purchase a monthly or weekly travel pass for public transportation?": "visit ticket counters, use automated ticket machines, or check online for options to purchase travel passes.",
    "are there bicycle rental options near my university for commuting?": "check with your university or local bike rental services for information on bicycle rentals near your campus.",
    "what's the procedure for obtaining an international driving license in the uk?": "apply for an international driving license in your home country before coming to the uk if you plan to drive.",
    "how do i pay for parking if i bring my car to the uk for studying?": "use parking meters, pay-and-display machines, or mobile apps to pay for parking in designated areas.",
    "what's the process for getting a uk driver's license as an international student?": "review the uk's driver's license requirements and take the necessary tests if you plan to drive in the uk.",
    "how to find safe and well-lit walking routes from my accommodation to the university?": "ask locals or use navigation apps to find well-lit walking routes, and avoid poorly lit areas at night.",
    "what should i do if i lose my travel card or oyster card for public transportation?": "report the loss immediately to your transportation provider to block the card and request a replacement.",
    "how to handle public transportation during holidays or weekends when schedules may change?": "check updated schedules on transportation websites or apps, and plan your trips accordingly during holidays or weekends.",
    "how do i access the night bus or late-night transportation options in the uk?": "check with local transportation providers for information on night buses or late-night services available in your area.",
    "what are the options for long-distance travel within the uk for weekend trips or holidays?": "consider taking trains or coaches for long-distance travel within the uk, and book tickets in advance for savings.",
    "how to handle public transportation strikes or disruptions that might affect my commute?": "stay informed about possible strikes or disruptions, and have backup transportation plans when needed.",
    "what's the procedure for getting a refund for delayed or canceled train or bus services?": "contact the transportation provider or check their website for information on refund procedures due to delays or cancellations.",
    "how to ensure safety while using public transportation, especially during late hours?": "stay alert, travel with friends if possible, and use well-lit stops and stations when using public transportation late at night.",
     "how can I find out about local bus routes and schedules in my city?": "Check with local transportation authorities or use transportation apps to access bus route and schedule information.",
    "what's the easiest way to get from the airport to my university or accommodation?": "Consider taking airport shuttles, taxis, or pre-booked airport transfers for a convenient journey to your destination.",
    "how do I apply for a railcard to save on train travel in the UK?": "Apply for a railcard online or at a train station to enjoy discounts on train fares across the UK.",
    "what's the procedure for renting a bicycle for daily commuting within the city?": "Visit local bike rental shops or use bike-sharing services in your city for convenient daily commuting.",
    "are there carpooling or ride-sharing options available for daily transportation?": "Explore ride-sharing apps and platforms like Uber or local carpooling services for daily transportation.",
    "what's the best way to handle transportation during bad weather conditions like snow or heavy rain?": "Check transportation advisories, plan for extra travel time, and consider alternative transportation options during bad weather.",
    "how can I get information about accessibility and accommodations for individuals with disabilities on public transportation?": "Contact local transportation authorities or visit their websites for details on accessible services.",
    "what's the policy for bringing a bicycle or large luggage on buses or trains?": "Review the transportation provider's policies regarding bicycles and large luggage, and inquire about specific rules.",
    "how can I stay updated on any changes or disruptions in the public transportation system?": "Subscribe to transportation alerts, follow transportation providers on social media, or check their websites for updates.",
    "what should I do if I need to travel late at night and taxis are not readily available?": "Plan your journey in advance, use ride-sharing apps, or consider booking a taxi in advance for late-night travel.",
    "how can I access student discounts on long-distance bus or coach travel in the UK?": "Look for student discounts offered by bus and coach companies, and book tickets using your student ID for savings.",
    "what's the procedure for applying for a disabled person's bus pass or travel card in the UK?": "Contact your local council or transportation authority to inquire about applying for a disabled person's travel pass.",
    "how do I pay for parking at university campuses or public parking areas near my accommodation?": "Use designated parking machines or mobile apps to pay for parking at university campuses or public parking areas.",
    "what's the best way to handle transportation during major events or festivals in the city?": "Plan your travel routes in advance, expect delays, and consider using public transportation to avoid traffic congestion.",
    "how can I find out about local bike lanes and cycling routes for safer commuting?": "Check with local authorities or use cycling apps to find information on bike lanes and cycling routes in your city.", 
     "what's the cost of a monthly public transportation pass for students in the uk?": "the cost varies by location. check with your local transportation authority for student pass pricing.",
    "how can i find the nearest train or bus station to my accommodation?": "use navigation apps or ask locals for directions to the nearest train or bus station.",
    "what are the options for purchasing single-ride tickets for buses and trains in my city?": "you can typically buy single-ride tickets at stations, onboard buses, or through mobile apps.",
    "how early should i arrive at the train or bus station to ensure i don't miss my ride?": "arrive at least 10-15 minutes before your scheduled departure time to allow for ticketing and boarding.",
    "how to handle lost property or items left on public transportation in the uk?": "contact the lost property office of the transportation provider to inquire about lost items.",
    "what's the best way to find a reliable taxi service for late-night transportation?": "use reputable taxi apps or services recommended by locals for late-night transportation.",
    "how can i track the real-time arrival of buses and trains in my city?": "download transportation apps or visit transportation authority websites for real-time tracking information.",
    "what's the procedure for applying for a disabled parking permit in the uk?": "contact your local council or visit their website for information on applying for a disabled parking permit.",
    "how to handle situations where i need to transfer between different modes of public transportation?": "plan your transfers in advance, allowing enough time between connections, and follow signs and announcements at stations.",
    "are there any restrictions on using public transportation with a bicycle or large luggage during peak hours?": "check with the transportation provider for any restrictions on carrying bicycles or large items during peak hours.",
    "how to handle transportation when traveling to other cities in the uk for vacations or holidays?": "consider using trains, coaches, or car rentals for intercity travel, and book tickets in advance for convenience.",
    "what's the best way to learn about special discounts or promotions on public transportation for students?": "check transportation authority websites, social media pages, or inquire at local transportation offices for information on student discounts.",
    "how can i stay safe while waiting at bus stops or train stations, especially during the evening?": "stay in well-lit areas, avoid distractions, and be cautious of your surroundings while waiting for public transportation in the evening.",
    "what's the procedure for requesting a refund for a canceled or missed flight from the uk to another city?": "contact the airline you booked with for information on requesting a refund for canceled or missed flights.",
    "how can i find bicycle repair shops or services near my accommodation in case of bike issues?": "search online for local bicycle repair shops or ask locals for recommendations on nearby repair services.", 
            
    
    # 3. Healthcare
    "medical services": "access to medical services in the uk covers a wide range of healthcare providers, including general practitioners (gps), specialists, and clinics.",
    "health services": "health services in the uk encompass preventive care, vaccinations, screenings, and treatments for various medical conditions through the national health service (nhs).",
    "nhs access": "access to the nhs in the uk is typically through gp registration, allowing individuals to receive primary healthcare services and referrals to specialists when needed.",
    "doctor": "consulting a doctor in the uk involves making an appointment with a local gp, who can address general health concerns and provide referrals to specialists.",
    "clinic": "clinics in the uk offer specialized services, including sexual health clinics, family planning clinics, and walk-in centers for non-emergency medical issues.",
    "hospital": "hospitals in the uk provide inpatient and outpatient care, with emergency departments for urgent medical attention and elective procedures for scheduled treatments.",
    "health insurance": "health insurance in the uk is available for those seeking additional coverage beyond the nhs, often covering private consultations, treatments, and faster access to specialists.",
    "gp registration": "gp registration is essential for accessing nhs healthcare services, allowing individuals to choose a local doctor's practice and receive personalized care.",
    "emergency services": "emergency services in the uk include calling 999 or 112 for immediate assistance from paramedics, police, or the fire brigade during life-threatening situations.",
    "medical care": "comprehensive medical care in the uk ensures that individuals receive prompt attention, diagnosis, and treatment for various health conditions within the nhs.",
    "how does healthcare work in the uk?": "healthcare in the uk is primarily provided through the national health service (nhs). it's a publicly funded system that offers a wide range of medical services to residents and eligible international students. to access healthcare, you should register with a local gp (doctor) and obtain an nhs number. in emergencies, you can visit an nhs hospital. it's essential to familiarize yourself with the nhs and how to access its services.",
    "healthcare for international students": "international students studying in the uk may be eligible for nhs healthcare services, but it's important to understand the specific eligibility criteria. to access healthcare, you should register with a local gp (doctor) and obtain an nhs number. additionally, consider obtaining health insurance to cover any medical expenses not included in nhs services. it's essential to proactively manage your healthcare needs while studying in the uk.",
    "national health service (nhs)": "the national health service (nhs) is the primary healthcare provider in the uk. it offers a wide range of medical services, including doctor's appointments, hospital care, and emergency services, to residents and eligible international students. to access nhs healthcare, you should register with a gp (doctor) in your local area. you will receive an nhs number, which is essential for booking appointments and accessing services. the nhs plays a crucial role in ensuring the well-being of people in the uk.",
    "healthcare in the uk": "healthcare in the uk is provided primarily through the national health service (nhs), a publicly funded healthcare system. it offers comprehensive medical services, including general practitioner (gp) visits, hospital care, and emergency treatment. registering with a gp in your local area is the first step to accessing healthcare services. while nhs services are available to residents and eligible international students, it's important to be aware of any additional health insurance coverage you may need to fully meet your healthcare needs during your stay in the uk.",
    "can you explain how to register with a gp in the uk?": "registering with a gp (general practitioner) in the uk is essential for accessing healthcare services. to register, you'll need to find a local gp practice, complete a registration form, and provide proof of your identity and address. once registered, you can book appointments for medical consultations.",
    "what healthcare services are covered by the nhs for international students?": "the nhs provides a wide range of healthcare services, including doctor's appointments, hospital treatment, emergency care, and specialist services. international students may be eligible for these services, but it's important to understand the specific coverage and any potential costs.",
    "do i need health insurance as an international student in the uk?": "while international students in the uk may access nhs healthcare services, it's advisable to consider obtaining health insurance. health insurance can cover additional medical expenses, such as dental care and repatriation, which may not be included in nhs services.",
    "how can i find a nearby hospital or medical facility in the uk?": "if you require medical assistance or need to locate a hospital or medical facility in the uk, you can use online resources, medical directories, or contact your gp for guidance. in emergencies, call 999 for immediate assistance.",
    "what should i do in case of a medical emergency in the uk?": "in a medical emergency in the uk, dial 999 for immediate assistance. emergency services will provide guidance and dispatch help as needed. it's important to stay calm and provide necessary information when making the call.",
    "what vaccinations or health precautions should i take before traveling to the uk?": "before traveling to the uk, it's essential to check if any vaccinations or health precautions are required or recommended. consult your healthcare provider or a travel clinic for guidance on vaccinations and health preparations.",
    "are there any specific health services available for mental health support in the uk?": "the uk offers mental health support services through the nhs. you can discuss mental health concerns with your gp, who can provide guidance and referrals to specialists or mental health services.",
    "can i access contraception and sexual health services in the uk as an international student?": "yes, international students in the uk can access contraception and sexual health services through the nhs. you can visit a local sexual health clinic or discuss your needs with your gp for appropriate care and advice.",
    "how can i see a doctor in the uk if i get sick?": "you can register with a local gp (general practitioner) and schedule an appointment.",
    "what should i do in case of a medical emergency in the uk?": "dial 999 for immediate help in case of a medical emergency.",
    "do i need health insurance as a student in the uk?": "while the nhs covers most healthcare, having health insurance is recommended for extra coverage.",
    "how do i get a european health insurance card (ehic) for healthcare in the uk and europe?": "apply for an ehic through your home country's health authority before coming to the uk.",
    "how can i find mental health support as a student in the uk?": "contact your university's counseling center or student support services for mental health support.",
    "how do i refill my prescription medications in the uk?": "visit your gp for prescription refills, and they'll provide your medications.",
    "what's the cost of medical treatments for international students in the uk?": "most treatments are covered by the nhs, but check with your gp for any additional charges.",
    "how can i find a dentist or eye doctor in the uk?": "search online or ask your gp for recommendations to find a dentist or optometrist.",
    "what should i do if i have symptoms of a common illness like a cold or flu?": "rest and stay hydrated. if symptoms persist, see your gp.",
    "how do i handle medical expenses and insurance claims for non-nhs treatments?": "submit claims to your health insurance provider and keep records of expenses for reimbursement.",
    "where can i get vaccines in the uk?": "visit a local clinic or your gp for vaccinations.",
    "how can i get a doctor's note for an illness?": "contact your gp to request a doctor's note for your illness.",
    "what do i do if i lose my ehic card in the uk?": "contact your home country's health authority for a replacement ehic card.",
    "how can i access after-hours medical care in the uk?": "dial 111 for non-emergency medical help after hours.",
    "where can i find information about nhs services near me?": "search online or ask your gp for information on nhs services in your area.",
    "what's the process for registering with a dentist in the uk?": "contact a local dentist to inquire about registration procedures.",
    "how can i get my prescription medications from a pharmacy in the uk?": "take your gp's prescription to a local pharmacy to obtain your medications.",
    "what should i do if i need a medical certificate for university-related reasons?": "discuss your need for a medical certificate with your gp.",
    "where can i find free or low-cost sexual health services in the uk?": "visit a sexual health clinic or check with your university for information on free or low-cost services.",
    "how do i obtain medical records for a visa application?": "request your medical records from your gp or healthcare provider.",
    "what's the procedure for getting a referral to a specialist in the uk?": "consult your gp, and they can refer you to a specialist if necessary.",
    "how can i access birth control methods in the uk as a student?": "visit a sexual health clinic or consult your gp for information on birth control options.",
        
        
    # 4. Jobs
    "work opportunities": "work opportunities in the uk range from part-time jobs for students to full-time positions in various industries, providing valuable work experience.",
    "part-time employment": "part-time employment is common for students, with options like retail, hospitality, and tutoring, allowing flexibility for studies.",
    "student jobs": "student jobs in the uk include roles on campus, such as library assistants and student ambassadors, as well as off-campus opportunities.",
    "internships": "internships in the uk offer practical experience in fields like finance, technology, and marketing, helping students build their careers.",
    "career": "building a career in the uk involves planning, skill development, and networking, with opportunities for growth in a competitive job market.",
    "job market": "the uk job market is diverse, with sectors like finance in london, technology in manchester, and creative industries in edinburgh, providing options for various career paths.",
    "work permits": "work permits in the uk depend on nationality, but international students can often work part-time during studies and full-time during holidays.",
    "workplace": "workplaces in the uk prioritize work-life balance, inclusivity, and career development, with options for remote work and flexible hours in some industries.",
    "job search": "job search strategies in the uk include online job boards, career fairs, networking events, and career services at universities.",
    "employment advice": "employment advice for students includes resume building, interview preparation, and seeking guidance from career advisors to enhance job prospects.",
    "can international students work in the uk?": "international students in the uk can work part-time during their studies and full-time during breaks. ensure you have the required permissions and a national insurance number.",
    "part-time jobs for students": "exploring part-time job opportunities can be beneficial for students in the uk. international students are generally allowed to work part-time during their studies, but it's essential to understand the rules and regulations.",
    "student employment rules": "understanding the employment rules for international students is crucial. while you can typically work part-time during your studies, it's important to check your visa conditions and obtain a national insurance number if required.",
    "where can i look for part-time jobs as an international student in the uk?": "finding part-time jobs can be facilitated by checking university job boards, online job search websites, and local businesses. networking and seeking advice from your university's career services can also be helpful.",
    "are there websites specifically for international students to find employment opportunities in the uk?": "several websites and job portals cater to international students seeking employment in the uk. these platforms often list job opportunities suitable for students and provide resources for job seekers.",
    "can international students find full-time employment in the uk after completing their studies?": "after completing their studies, international students in the uk can explore options for full-time employment. this typically involves applying for a work visa, such as the tier 2 (general) visa. be sure to research the requirements and application process.",
    "what are some tips for preparing for a job interview in the uk?": "preparing for a job interview in the uk involves researching the company, practicing common interview questions, dressing professionally, and demonstrating your skills and enthusiasm. utilize university career services for interview preparation assistance.",
    "how can i create an effective cv (curriculum vitae) for job applications in the uk?": "crafting a strong cv is essential for job applications. include your education, work experience, skills, and achievements. there are online resources and templates available to help you create an impressive cv. consider seeking guidance from university career advisors as well.",
    "can international students work in the uk?": "international students in the uk can work part-time during their studies and full-time during breaks. ensure you have the required permissions and a national insurance number.",
    "part-time jobs for students": "exploring part-time job opportunities can be beneficial for students in the uk. international students are generally allowed to work part-time during their studies, but it's essential to understand the rules and regulations.",
    "student employment rules": "understanding the employment rules for international students is crucial. while you can typically work part-time during your studies, it's important to check your visa conditions and obtain a national insurance number if required.",
    "how to find part-time jobs as a student?": "check job search websites, university career centers, and local job boards for part-time job listings.",
    "what are the rules for student work hours?": "students can typically work up to 20 hours per week during term and full-time during holidays.",
    "how to create a cv or resume for job applications?": "use online templates to create a cv highlighting your skills, education, and work experience.",
    "how to prepare for job interviews?": "practice common interview questions, research the company, and dress professionally for interviews.",
    "where to get info about work visas and post-study work?": "visit official government websites or consult with your university for visa and work opportunities info.",
    "how to open a bank account for salary?": "contact local banks for student account options and required documentation for account opening.",
    "how to obtain a national insurance number (nin) for employment?": "apply for a national insurance number through government websites or local offices.",
    "how to handle income tax as a student?": "check with tax authorities for information on tax obligations and filing as a student.",
    "where to find internships or work placements related to my field?": "explore university resources and online job portals for internship opportunities.",
    "what's the minimum wage for part-time jobs and does it vary by age?": "minimum wage rates vary, so check government websites for current rates.",
    "how to balance part-time work with studies effectively?": "create a schedule prioritizing studies, communicate availability to employers, and seek flexible job options.",
    "what to do if i face workplace issues or conflicts?": "discuss issues with your employer and seek guidance from university student support services if needed.",
    "how to network with potential employers or industry professionals?": "attend career fairs, join student organizations, and use professional networking platforms.",
    "what are the steps to apply for work-related visas after graduation?": "review visa application processes and eligibility criteria for post-graduation work visas.",
    "how to get assistance with job search and career development?": "utilize career resources and services offered by your university.",
    "how to write a cover letter for job applications?": "follow cover letter templates and highlight your qualifications and enthusiasm for the position.",
    "what are common job search websites to find part-time jobs?": "popular job search websites include indeed, linkedin, and glassdoor.",
    "how to improve my job interview skills?": "practice with mock interviews, seek feedback, and research common interview questions.",
    "where to find information about job fairs or career events on campus?": "check with your university's career center or bulletin boards for event announcements.",
    "how to build a professional linkedin profile for job opportunities?": "upload a professional photo, include relevant skills and experience, and connect with professionals.",
    "what are common student job roles or industries to explore?": "common student job roles include retail, hospitality, tutoring, and internships in various fields.",
    "how to inquire about job openings when visiting local businesses?": "ask to speak with a manager and express your interest in available job opportunities.",
    "what should i wear for job interviews or career fairs?": "dress professionally in business attire to make a positive impression.",
    "how to create an effective online job application profile?": "fill out application forms accurately, provide a professional email, and include relevant details.",
    "where to get references for job applications if i don't have work experience?": "consider using professors, advisors, or volunteer coordinators as references.",
    "how to approach part-time jobs that require previous experience?": "apply for entry-level positions or internships to gain relevant experience.",
    "what to do if i want to switch part-time jobs while studying?": "apply for new positions and, if offered, provide notice to your current employer.",
    "how to handle a job offer if i receive more than one?": "compare offers, consider factors like pay and location, and make an informed decision.",
    "what are the benefits of part-time jobs for students?": "part-time jobs can provide income, work experience, and opportunities to develop skills.",
    "how to create a study-work-life balance as a student with a part-time job?": "plan your schedule, prioritize tasks, and communicate with your employer about availability.",
    "how to write a cover letter?": "follow templates and highlight qualifications and enthusiasm.",
    "where to find part-time jobs?": "check popular job search websites.",
    "how to improve job interview skills?": "practice and research common questions.",
    "where to find job fairs on campus?": "check with the career center.",
    "how to build a linkedin profile?": "use a professional photo and include skills.",
    "common student job roles?": "retail, hospitality, tutoring, internships.",
    "how to inquire about job openings in person?": "speak with a manager directly.",
    "what to wear for interviews?": "dress professionally in business attire.",
    "create an effective online job application profile?": "fill out forms accurately and provide a professional email.",
    "where to get job references without experience?": "ask professors or advisors.",
    "approach jobs requiring experience?": "apply for entry-level positions or internships.",
    "how to switch part-time jobs?": "apply for new positions and provide notice.",
    "handle multiple job offers?": "compare and make an informed decision.",
    "benefits of part-time jobs for students?": "income, experience, skill development.",
    "create a study-work-life balance with a job?": "plan, prioritize, and communicate with your employer.",  
        
    
    # 5. Emergency
    "urgent help": "in emergencies in the uk, one can seek urgent help by calling 999 or 112 for immediate assistance from police, fire brigade, or ambulance services.",
    "crisis": "during a crisis in the uk, it's essential to stay calm and call emergency services, which are available 24/7 to address various crises and emergencies.",
    "urgent care": "urgent care centers in the uk provide medical assistance for non-life-threatening issues when immediate attention is needed outside regular gp hours.",
    "police assistance": "for police assistance in the uk, dial 999 or 112 to report crimes, emergencies, or suspicious activities, and the police will respond promptly.",
    "fire brigade": "the fire brigade in the uk responds to fires, accidents, and hazardous incidents, with emergency numbers 999 or 112 for quick assistance.",
    "ambulance": "in medical emergencies, calling 999 or 112 for an ambulance in the uk ensures swift access to paramedics and healthcare services.",
    "safety": "ensuring safety in emergencies in the uk involves following safety guidelines, evacuating if necessary, and relying on emergency services for assistance.",
    "first aid": "knowing first aid basics can be crucial in emergencies; consider taking a first aid course to provide initial assistance before professional help arrives.",
    "accident": "in the event of an accident in the uk, call emergency services immediately, and remember that providing accurate location details is essential for a swift response.",
    "hazard": "identifying hazards and taking precautions can prevent emergencies; be aware of safety measures in place and report hazards to the appropriate authorities.",
    "what to do in case of an emergency?": "in case of an emergency in the uk, dial 999 for police, fire, or medical assistance. it's important to know the local emergency services number.",
    "emergency contact in the uk": "having emergency contacts in the uk is crucial. save important numbers on your phone, including 999 for emergencies, and know the locations of the nearest hospitals and police stations.",
    "safety tips": "safety is paramount. stay informed about safety tips and practices, such as keeping your phone charged, sharing your whereabouts with someone you trust, and following local safety guidelines.",
    "what to do in case of an emergency?": "in case of an emergency in the uk, dial 999 for police, fire, or medical assistance. it's important to know the local emergency services number.",
    "emergency contact in the uk": "having emergency contacts in the uk is crucial. save important numbers on your phone, including 999 for emergencies, and know the locations of the nearest hospitals and police stations.",
    "safety tips": "safety is paramount. stay informed about safety tips and practices, such as keeping your phone charged, sharing your whereabouts with someone you trust, and following local safety guidelines.",
    "how can i register for emergency alerts in the uk?": "registering for emergency alerts is a proactive way to stay informed about potential hazards. look for official websites or government platforms where you can sign up to receive alerts and updates.",
    "where should i register for emergency notifications as an international student in the uk?": "international students in the uk can typically register for emergency notifications through university channels or local government websites. check with your university's emergency services or international student office for guidance.",
    "what are some websites or resources to access emergency preparedness information in the uk?": "there are various websites and resources available for accessing emergency preparedness information in the uk. look for government websites, local emergency management agencies, and university resources to stay informed about safety measures and emergency procedures.",
    "how can i prepare an emergency kit for unexpected situations in the uk?": "creating an emergency kit is a wise precaution. learn what essential items to include in your kit, such as water, non-perishable food, a flashlight, first-aid supplies, and important documents.",
    "what should international students know about fire safety in the uk?": "fire safety is crucial. familiarize yourself with fire evacuation procedures, fire alarms, and fire extinguisher locations in your accommodation. follow safety guidelines to prevent fire incidents.",
    "are there specific safety precautions for traveling within the uk, especially during adverse weather conditions?": "traveling safely in the uk, especially during adverse weather, requires careful planning. stay informed about weather forecasts, road conditions, and transportation disruptions. always prioritize safety when traveling.",

    # 6. Shopping
    "what are the popular shopping destinations in the uk?" : "the uk offers a wide range of shopping destinations, including famous streets like oxford street in london, shopping centers like westfield, and vibrant markets like camden market. explore these popular spots for a great shopping experience.",
    "how can i find the best deals and discounts while shopping in the uk?" : "finding the best deals and discounts involves using loyalty cards, checking for sales, and utilizing discount codes. additionally, consider shopping during sales events like black friday or using cashback apps for savings.",
    "what are the opening hours of shops in the uk, especially on weekends?" : "shop opening hours can vary, but many stores are open from around 9 am to 5 or 6 pm. on weekends, some shops may have shorter hours, so it's essential to check the specific timings of the stores you plan to visit.",
    "are there any tax-free shopping options for international visitors in the uk?" : "international visitors can often enjoy tax-free shopping in the uk. look for shops displaying the tax-free shopping sign, and follow the necessary procedures to claim your vat refund upon departure.",
    "what are some popular uk brands and products that are great for souvenirs?" : "the uk offers a wide range of unique brands and products suitable for souvenirs. consider items like british tea, biscuits, luxury chocolates, and fashion items from renowned british designers.",
    "can you recommend eco-friendly and sustainable shopping options in the uk?" : "eco-conscious shoppers can explore sustainable and eco-friendly stores in the uk. look for shops specializing in ethically sourced clothing, organic products, and eco-friendly household items.",
    "how do i navigate online shopping and delivery services in the uk?" : "online shopping is popular in the uk. learn how to navigate e-commerce websites, choose reliable online retailers, and understand the delivery options and costs for your purchases.",
    "what should i know about shopping etiquette and customs in the uk?" : "understanding shopping etiquette in the uk includes queuing (standing in line), being polite to staff, and respecting store policies. it's also customary to greet with 'please' and 'thank you' during transactions.",
    "what are the best places to buy second-hand or vintage items in the uk?" : "the uk has a thriving market for second-hand and vintage items. explore vintage shops, charity stores, and flea markets for unique finds and sustainable shopping options.",
    "what are some money-saving tips for international students studying in the uk?" : "international students can save money by budgeting wisely, using student discounts, cooking meals at home, and taking advantage of free or low-cost entertainment options.",
    "when are the best times to find sales like black friday and christmas sales in the uk for student discounts?" : "black friday and christmas sales in the uk offer excellent opportunities for student discounts. these sales typically occur in late november and december, providing substantial savings on various products.",
    "where can international students in the uk find affordable groceries for their daily needs?" : "international students can save on groceries by shopping at budget-friendly supermarkets like aldi, lidl, and morrisons. additionally, consider using loyalty cards for discounts.",
    "how can students find necessary stationary items like textbooks and school supplies for affordable prices in the uk?" : "students can save on textbooks by buying second-hand or digital versions. look for student discounts at bookstores and explore online platforms for affordable school supplies.",
    "where can i buy groceries near my university accommodation?": "check nearby supermarkets or local grocery stores for your grocery needs.",
    "what are some popular clothing stores in the area for students?": "explore local malls or shopping districts for clothing shops popular among students.",
    "how to find affordable furniture or home goods for my student accommodation?": "look for budget-friendly furniture stores, thrift shops, or online marketplaces.",
    "where to buy textbooks or academic supplies for my courses?": "visit the university bookstore or online platforms for textbooks and academic supplies.",
    "how to take advantage of student discounts while shopping?": "show your student id or register for student discount programs offered by stores.",
    "are there any secondhand or thrift stores where i can find bargains?": "search for local thrift shops or browse online platforms for secondhand items.",
    "how to find the best deals and sales on electronics and gadgets as a student?": "check for seasonal sales, subscribe to newsletters, and compare prices online.",
    "what's the procedure for returning or exchanging purchased items at local stores?": "review the store's return policy and keep receipts for hassle-free returns or exchanges.",
    "how to handle online shopping and delivery for items i purchase?": "use reliable online retailers and provide accurate delivery information for smooth transactions.",
    "what are some tips for budget-friendly shopping as a student?": "set a budget, make a shopping list, and look for discounts and coupons.",
    "where to buy kitchenware and cooking utensils for my student accommodation?": "explore stores specializing in kitchenware or consider online shopping for convenience.",
    "how to find affordable and stylish clothing for special occasions or events?": "look for discount clothing stores or consider renting formal wear when needed.",
    "what should i do if i receive a damaged or incorrect item when shopping online?": "contact the seller or retailer's customer service for assistance with returns or replacements.",
    "how can i sell or donate items i no longer need while in the uk?": "consider online marketplaces, local charity shops, or donation centers for unwanted items.",
    "how to stay safe while shopping in the area, especially when making online transactions?": "shop from reputable sources, use secure payment methods, and be cautious of phishing scams.",
    "where to buy cheap groceries nearby?": "check local supermarkets for affordable options.",
    "best stores for student fashion?": "explore nearby shops for stylish student clothing.",
    "find affordable furniture for students?": "look at budget-friendly furniture stores or online.",
    "where to get textbooks and school supplies?": "visit the university bookstore or shop online.",
    "how to get student discounts when shopping?": "show your student id or join student discount programs.",
    "find secondhand or thrift stores?": "search for thrift shops or browse online for bargains.",
    "get deals on electronics and gadgets?": "check sales, subscribe to newsletters, and compare prices online.",
    "returning or exchanging purchased items?": "follow the store's return policy and keep receipts.",
    "handle online shopping and delivery?": "use reliable online retailers and provide accurate delivery details.",
    "tips for budget-friendly shopping?": "set a budget, make a list, and use discounts or coupons.",
    "buy kitchenware and cooking utensils?": "visit kitchenware stores or shop online for convenience.",
    "find affordable clothing for special occasions?": "look for discounts or consider renting formal wear.",
    "dealing with damaged or incorrect online orders?": "contact customer service for assistance.",
    "sell or donate unwanted items in the uk?": "consider online marketplaces or local charity shops.",
    "stay safe while shopping, especially online?": "shop from trusted sources, use secure payments, and be cautious of scams.",  
        

    # General
    "what are some essential items to pack when traveling as an international student to the uk?" : "preparing a checklist of essential items is crucial when traveling as an international student. items may include travel documents, clothing suitable for the uk climate, adapters for electrical devices, and important personal belongings.",
    "what are the accommodation options for international students in the uk, and how can i choose the right one?" : "international students in the uk can choose from various accommodation options, including university dorms, private rentals, and shared housing. consider factors like budget, location, and preferences when selecting the right accommodation.",
    "how can i find part-time jobs or internships related to my field of study as an international student in the uk?" : "finding part-time jobs or internships related to your field of study involves searching job boards, networking with professors and peers, and using university career services for guidance.",
    "what are the cultural exchange opportunities available for international students in the uk?" : "international students can engage in cultural exchange through various activities, such as joining student clubs and societies, attending cultural events, and participating in language exchange programs.",
    "how can i make the most of my university experience as an international student in the uk?" : "maximizing your university experience involves actively participating in academic and extracurricular activities, seeking support when needed, and making connections with fellow students and faculty members.",
    "can you provide information about student visa requirements and application procedures for studying in the uk?" : "understanding student visa requirements is essential for international students. learn about visa categories, eligibility criteria, and the application process to ensure a smooth transition to the uk.",
    "where can i find information about scholarships and financial aid options for international students in the uk?" : "searching for scholarships and financial aid opportunities requires exploring university websites, government programs, and international scholarship databases to secure financial support for your studies.",
    "what are some tips for managing homesickness and adapting to life as an international student in a new country?" : "dealing with homesickness is a common challenge for international students. tips for adaptation include staying connected with loved ones, exploring the local culture, and seeking support from university counseling services.",
    "what are the top universities in the uk for my field of study?" : "i can provide information about the top universities in the uk based on your field of study and preferences.",
    "how can i open a bank account as an international student in the uk, and what documents do i need?" : "opening a bank account in the uk is essential. i can guide you through the process and let you know what documents are required.",
    "what are some cultural differences and etiquettes i should be aware of as an international student in the uk?" : "understanding cultural differences and etiquettes can help you adapt better. i can provide insights into uk culture and norms.",
    "where can i find affordable and reliable transportation options in the uk as a student?" : "navigating transportation is important. i can suggest budget-friendly transportation options and provide information about student travel cards.",
    "how do i apply for a national insurance number (nin) as an international student, and why is it important?" : "applying for a national insurance number is crucial for working in the uk. i can guide you through the application process and explain its importance.",
    "are there any health insurance requirements for international students in the uk, and how can i access healthcare services?" : "understanding health insurance and accessing healthcare services is vital. i can provide information on insurance requirements and the uk's healthcare system.",
    "can you suggest resources for improving my english language skills while studying in the uk?" : "improving your english skills is a great idea. i can recommend language courses, apps, and language exchange opportunities.",
    "what are the best student discounts and deals available in the uk for international students?" : "i can provide information about various student discounts and deals that can help you save money during your studies in the uk.",
    

    # money-saving
    "how can i save money in the uk?": "you can save money by using public transport, cooking your meals, and shopping for groceries wisely. also, consider buying second-hand items.",
    "money-saving tips": "saving money in the uk is possible by budgeting, using student discounts, and taking advantage of free or low-cost attractions.",
    "ways to save money": "consider using budgeting apps to manage your finances and track your expenses. this can help you identify areas where you can save.",
    "budget-friendly living tips in the uk": "for affordable entertainment, look for free events in your local area or discounts for students at cinemas and museums.",
    "advice for economical living": "when shopping, take advantage of sales, use price comparison websites, and consider buying in bulk for items you frequently use.",
    "how to manage finances as a student": "to reduce utility bills, ensure you turn off lights and appliances when not in use, and consider energy-efficient options.",
    "cost-saving strategies while studying": "cost-saving strategies for students include renting textbooks, sharing expenses with roommates, and taking advantage of student discounts on transportation and meals.",
    "tips for cheap shopping in the uk": "when shopping in the uk, consider using cashback apps, loyalty cards, and shopping during sale seasons to save money.",
    "affordable entertainment options": "look for free or low-cost entertainment options such as public parks, libraries, and community events.",
    "saving on utility bills": "to save on utility bills, consider energy-saving practices like using led bulbs, setting thermostats efficiently, and fixing any leaks or drafts in your home.",
    "how to cut down on daily expenses": "cutting down on daily expenses can be achieved by packing lunches, brewing your coffee, and avoiding unnecessary impulse purchases.",
    "effective budgeting techniques": "effective budgeting involves creating a budget plan, tracking your spending, setting financial goals, and making adjustments as needed.",
    "using discounts and offers effectively": "use discounts and offers effectively by researching deals, comparing prices, and avoiding impulse purchases just because something is on sale.",
    "cheap transportation options": "consider using public transportation, carpooling, or biking as cheap transportation alternatives in the uk to save on commuting costs.",
  
    # International Student FAQ
    "how to apply for a uk student visa": "to apply for a uk student visa, visit the official uk government website and follow the application process. make sure to prepare all required documents, including your offer letter, financial evidence, and passport details.",
    "opening a uk bank account as an international student": "to open a uk bank account as an international student, you'll typically need proof of enrollment, your passport, and proof of address. contact local banks near your university for specific requirements and to set up an appointment.",
    "finding affordable accommodation near my university": "finding affordable accommodation near your university is essential. check your university's accommodation services, student union, or websites like rightmove, zoopla, and spareroom for listings. consider on-campus housing too.",
    "dealing with culture shock in the uk": "experiencing culture shock is common. reach out to your university's international student support services for guidance and resources. connecting with fellow international students can also help.",
    "finding part-time jobs while studying in the uk": "you can find part-time jobs by checking university notice boards, online job portals, or visiting your university's career center for assistance. ensure you're aware of visa regulations regarding work.",
    "best mobile phone plans for international students": "several mobile providers offer plans for international students in the uk. look into companies like ee, o2, vodafone, and three. compare their offerings to find a suitable plan.",
    "extending a student visa for further studies": "to extend your student visa in the uk, apply before your current visa expires. review the official uk government website for extension requirements and begin the application process early.",
    "public transportation options and getting an oyster card in london": "public transportation in the uk is efficient. in london, get an oyster card for buses and trains by visiting the transport for london (tfl) website or a local tfl station.",
    "finding information on student discounts and deals": "you can find student discounts and deals on websites like studentbeans, unidays, and nus. many shops, restaurants, and cinemas in the uk offer discounts to students, so carry your student id.",
    "healthcare options for international students in the uk": "international students can access healthcare through the national health service (nhs) in the uk. register with a local gp (doctor) and consider health insurance for additional coverage.",
    "what documents are required for my student visa application?": "for your uk student visa application, you'll need documents like your cas (confirmation of acceptance for studies) from your university, proof of financial support, passport, and biometric information. check the official uk government website for a complete list.",
    "how can i prepare for my english language proficiency test, such as ielts or toefl?": "preparing for english language proficiency tests is essential. you can take preparatory courses, use study materials and practice tests, and consider enrolling in language classes at your university.",
    "finding english language support services at my university": "many uk universities offer english language support services, including language courses and conversation partners. contact your university's language center or international student office for information.",
    "what should i pack for my journey to the uk?": "when packing for your journey to the uk, include essentials like your passport, visa documents, academic certificates, and clothing suitable for the uk's weather. don't forget adapters for electrical outlets.",
    "navigating the uk's public transportation system": "the uk has an extensive public transportation system. familiarize yourself with local buses, trains, and the tube (in london) routes. you can use apps like google maps or citymapper for navigation.",
    "how to manage my finances while studying in the uk": "managing finances is important. create a budget, track expenses, and consider opening a uk bank account. take advantage of student discounts and cook meals at home to save money.",
    "understanding the uk's academic grading system": "the uk uses a different grading system. familiarize yourself with it, where a-levels, gcses, and degrees are graded from a* to u. your university will provide guidance on understanding your grades.",
    "joining student clubs and societies at my university": "joining clubs and societies is a great way to make friends. attend your university's freshers' fair or check their website for a list of student organizations and how to join them.",
    "exploring healthcare options and insurance for dental and eye care": "aside from the nhs, you may need insurance for dental and eye care. some universities offer health insurance plans for students. check with your university's health services for details.",
    "understanding uk classroom etiquette and academic expectations": "uk classrooms have unique etiquette. address professors as 'dr.' or 'professor,' participate in discussions, and meet assignment deadlines. attend orientation sessions to understand academic expectations.",
    "how to obtain a national insurance number (nin) in the uk": "to obtain a national insurance number (nin) in the uk, you'll need to apply through the department for work and pensions (dwp). follow their guidelines and complete the application process.",
    "what should i do if i lose my passport or visa documents while in the uk?": "if you lose your passport or visa documents in the uk, contact your country's embassy or consulate immediately. they can provide guidance on replacing your lost documents.",
    "how to make friends and connect with local students in the uk": "making friends with local students can enhance your cultural experience. attend social events, join clubs, and engage in activities on campus to connect with local students.",
    "finding part-time work during term breaks and holidays": "you can look for part-time work during term breaks and holidays to supplement your income. be sure to check visa regulations and work restrictions for international students.",
    "exploring options for student housing during term breaks": "if you need accommodation during term breaks, inquire with your university about options. some universities offer housing during holidays, or you can explore short-term rentals.",
    "accessing academic support services, including tutoring and writing centers": "universities often provide academic support services, including tutoring and writing centers. these resources can help you excel in your studies. contact your university's academic support offices for details.",
    "navigating the uk's healthcare system for non-emergencies": "for non-emergencies, use the nhs services. register with a local gp (doctor) for routine healthcare needs. in case of emergencies, dial 999 for immediate assistance.",
    "understanding the process of extending your student visa for employment after graduation": "if you plan to work in the uk after graduation, you may need to extend your student visa. explore options like the graduate route and consult your university's career services for guidance.",
    "tips for managing homesickness while studying abroad": "homesickness is normal. stay connected with family and friends through calls and video chats, explore your new surroundings, and engage in activities to combat homesickness.",
    "how to apply for a uk driving license as an international student": "applying for a uk driving license as an international student involves a process of theory and practical tests. review the driver and vehicle licensing agency (dvla) requirements and take driving lessons if necessary.",
    "managing stress and mental health while studying in a foreign country": "managing stress and mental health is crucial. your university likely offers counseling services. reach out to them if needed, and don't hesitate to talk to friends or family for support.",
    "how to obtain a uk mobile phone number and sim card": "getting a uk mobile phone number and sim card is easy. visit a local mobile store, provide identification, and choose a suitable plan. most providers offer prepaid and contract options.",
    "tips for dealing with british weather and seasonal changes": "the uk weather can vary. dress in layers, carry an umbrella, and be prepared for rain. check weather forecasts to plan activities accordingly.",
    "managing time effectively for coursework and assignments": "effective time management is essential. create a study schedule, prioritize tasks, and use productivity techniques like the pomodoro method to stay focused.",
    "how to report incidents of discrimination or harassment at university": "if you experience discrimination or harassment, universities have support systems in place. contact your university's student services or equalities office to report incidents and seek assistance.",
    "what to do if my student visa is about to expire, but i haven't completed my studies": "if your student visa is about to expire, but you haven't completed your studies, contact the uk visas and immigration (ukvi) office for guidance on visa extensions or transitioning to another visa category.",
    "exploring transportation options for weekend trips and exploring the uk": "for weekend trips and exploring the uk, consider train and coach travel. you can find discounted railcards for students that offer savings on fares for travel across the country.",
    "tips for effective communication with professors and academic staff": "effective communication with professors is important. attend office hours, ask questions during lectures, and use formal language in emails when addressing academic staff.",
    "understanding the uk's academic calendar and term dates": "familiarize yourself with the uk's academic calendar, including term dates, breaks, and exam periods. your university's academic calendar should have this information.",
    "finding part-time internships or work placements related to your field of study": "look for part-time internships or work placements in your field of study. check with your university's career services for opportunities and resources on preparing your resume and cover letter.",
    "how to deal with currency exchange and banking": "currency exchange and banking are straightforward. use atms to withdraw cash in gbp, and consider using international banks with branches in the uk for ease of transactions.",
    "exploring opportunities for volunteering and community engagement": "volunteering and community engagement are enriching experiences. your university or local organizations may offer volunteer opportunities. it's a great way to give back and meet people.",
    "what to do if my passport is expiring soon while studying in the uk": "if your passport is expiring soon while studying in the uk, contact your country's embassy or consulate for guidance on passport renewal. keep your documents up to date to avoid any issues.",
    "how to handle academic stress and exam preparation": "managing academic stress is crucial. create a study plan, seek help from professors or tutors if needed, and practice relaxation techniques like mindfulness or yoga.",
    "finding vegetarian and vegan food options in the uk": "the uk offers a variety of vegetarian and vegan food options. many restaurants and supermarkets have dedicated sections for plant-based foods. explore local markets for fresh produce.",
    "what to do in case of a medical emergency while in the uk": "in case of a medical emergency in the uk, dial 999 for immediate assistance. familiarize yourself with the nearest hospitals and clinics in your area.",
    "tips for successful group projects and collaborations with classmates": "group projects are common. communicate effectively with group members, set clear goals and deadlines, and distribute tasks evenly. regular meetings can ensure everyone stays on track.",
    "how to adapt to british etiquette and social norms": "adapting to british etiquette involves being polite, respecting personal space, and queuing (standing in line) patiently. observe local customs and be mindful of cultural differences.",
    "exploring options for student discounts on transportation and attractions": "take advantage of student discounts on transportation and attractions in the uk. many cities offer reduced fares for students on buses and trains. carry your student id to avail of discounts.",
    "navigating the uk's rental market and finding off-campus housing": "when searching for off-campus housing, use websites like rightmove, zoopla, or spareroom. attend property viewings, read lease agreements carefully, and consider housemates.",
    "how to stay safe and aware of personal security in the uk": "stay safe by being aware of your surroundings, avoiding poorly lit areas at night, and not sharing personal information with strangers. familiarize yourself with local emergency numbers.",
    "managing the cost of textbooks and course materials": "textbooks can be expensive. explore options like borrowing from the library, buying used books, or checking if digital versions are available. consider forming study groups to share resources.",
    "dealing with homesickness and staying connected with family and friends": "homesickness is common. stay connected with loved ones through video calls, send care packages from home, and engage in social activities to build a support network.",
    "how to register with a local doctor (gp) for healthcare services": "to register with a local doctor (gp) in the uk, visit a gp surgery near your residence. fill out registration forms and provide identification. registering is essential for routine healthcare.",
    "finding resources and support for lgbtq+ students in the uk": "the uk is lgbtq+ friendly. look for lgbtq+ societies at your university, explore local lgbtq+ community centers, and access support services if needed.",
}

# Dictionary to store conversation state for each user
conversation_states = {}

# Enhanced function to identify topic based on user input
def identify_topic(user_input):
    # Basic keyword mapping to topics
    keywords_to_topics = {
        "accommodation": "accommodation",
        "travel": "travel",
        "transport": "transport",
        "healthcare": "healthcare",
        "culture": "culture",
        "jobs": "employment",
        "emergency": "emergency",
        "postgraduate": "education",
        "admission": "admission",
        "scholarship": "scholarship",
        "deadlines": "deadlines",
    }

    user_input_lower = user_input.lower()
    for keyword, topic in keywords_to_topics.items():
        if keyword in user_input_lower:
            return topic
    return "general"

# Enhanced function to generate topic-based response
def generate_topic_response(topic, user_input):
    topic_responses = {
        "accommodation": "In the UK, accommodation options include university halls, private student halls, and private housing. Costs vary depending on location and housing type.",
        "travel": "Traveling in the UK offers options like buses, trains, or flights. For budget travel, consider coaches or advance train bookings.",
        "transport": "The UK has a comprehensive public transportation system, including buses, trains, trams, and the London Underground.",
        "healthcare": "International students in the UK are eligible for NHS services. Register with a local GP upon arrival.",
        "culture": "British culture includes a rich mix of historical traditions and modern influences.",
        "employment": "International students in the UK can work up to 20 hours per week during term time.",
        "emergency": "In an emergency in the UK, dial 999 for immediate assistance.",
        "education": "For postgraduate courses in the UK, you typically need a bachelor's degree, language proficiency, and sometimes work experience.",
        "admission": "Admission to UK universities requires meeting academic and language criteria.",
        "scholarship": "Scholarships in UK universities are available for academic excellence, sports abilities, or artistic talents.",
        "deadlines": "Application deadlines for UK universities vary. Generally, UCAS undergraduate applications are due in January.",
    }
    return topic_responses.get(topic, "I'm not sure how to respond to that. Can you provide more details or ask about something else?")

# Function to get a response with conversation state
def get_response_with_state(model, tokenizer, label_encoder, user_input, user_id):
    if user_id not in conversation_states:
        conversation_states[user_id] = {"step": 1, "topic": None}

    current_state = conversation_states[user_id]
    if current_state["step"] == 1:
        topic = identify_topic(user_input)
        conversation_states[user_id]["topic"] = topic
        conversation_states[user_id]["step"] = 2
        return f"Let's talk about {topic}. What specific information do you need?"
    elif current_state["step"] == 2:
        topic = current_state["topic"]
        response = generate_topic_response(topic, user_input)
        return response

# Load intents from intents.json
with open("intents.json") as file:
    intents_data = json.load(file)

# Function to load the pre-trained model
def load_model():
    return keras.models.load_model('chat_model')

# Function to load the tokenizer
def load_tokenizer():
    with open('tokenizer.pickle', 'rb') as handle:
        return pickle.load(handle)

# Function to load the label encoder
def load_label_encoder():
    with open('label_encoder.pickle', 'rb') as enc:
        return pickle.load(enc)

# Function to get a response from the chatbot model
def get_response(model, tokenizer, label_encoder, user_input):
    max_sequence_length = 20
    result = model.predict(keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([user_input]),
        truncating='post', maxlen=max_sequence_length))
    predicted_tag = label_encoder.inverse_transform([np.argmax(result)])

    for intent in intents_data['intents']:
        if intent['tag'] == predicted_tag:
            return np.random.choice(intent['responses'])

# Create the Dash application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# app layout
app.layout = dbc.Container([
    dbc.Row(
        dbc.Col(intro_message, width=12)  # Intro message at the top
    ),
    dbc.Row(
        dbc.Col(dcc.Textarea(id='user-input', placeholder="Enter your question here...", style={'width': '100%', 'height': '100px'}), width=12)
    ),
    dbc.Row(
        dbc.Col(html.Button('Submit', id='submit-button', n_clicks=0, className="mt-2"), width=12)
    ),
    dbc.Row(
        dbc.Col(html.Div(id='chat-history', style={'marginTop': 20, 'border': '1px solid #ddd', 'padding': '10px', 'height': '300px', 'overflowY': 'scroll'}), width=12)
    ),
    html.Div(id='session-id', style={'display': 'none'}, children='unique_user_id')
], fluid=True)


@app.callback(
    [Output('chat-history', 'children'),
     Output('user-input', 'value')],
    [Input('submit-button', 'n_clicks')],
    [State('user-input', 'value'),
     State('chat-history', 'children'),
     State('session-id', 'children')]
)
def update_output(n_clicks, user_input, chat_history, user_id):
    if n_clicks > 0 and user_input:
        # Check if the input mentions a country other than the UK
        mentioned_country = country_in_query(user_input)
        if mentioned_country and mentioned_country.lower() != 'united kingdom':
            response = "I can only provide details for the UK, not about other countries."
        else:
            # Check for a keyword-based response
            keyword_response = get_keyword_response(user_input, keyword_responses)
            if keyword_response:
                response = keyword_response
            else:
                # If no keyword match, use the trained model
                chatbot_model = load_model()
                chatbot_tokenizer = load_tokenizer()
                chatbot_label_encoder = load_label_encoder()
                response = get_response(chatbot_model, chatbot_tokenizer, chatbot_label_encoder, user_input)

        # Update chat history
        new_chat = html.Div([
            html.P("User: " + user_input, style={'color': 'blue'}),
            html.P("Chatbot: " + response, style={'color': 'green'})
        ])

        chat_history = chat_history or []
        chat_history.append(new_chat)

        return chat_history, ""
    return chat_history, user_input



if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
